In [1]:
import tensorflow.compat.v2 as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import time
keras = tf.keras
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\0\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\0\test'
checkpoint_path = r'C:\Users\24306\Desktop\RegNetY008\0\RegNetY008.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(train_data)



[37.0, 40.0, 16.0, 96.0, 95.0, 39.0, 7.0, 32.0, 80.0, 47.0, 13.0, 80.0, 47.0, 86.0, 95.0, 10.0, 13.0, 7.0, 55.0, 96.0, 93.0, 100.0, 96.0, 39.0, 80.0, 93.0, 93.0, 47.0, 30.0, 84.0, 52.0, 47.0, 55.0, 37.0, 37.0, 80.0, 45.0, 93.0, 100.0, 47.0, 66.0, 84.0, 7.0, 20.0, 95.0, 52.0, 96.0, 52.0, 100.0, 66.0, 84.0, 45.0, 86.0, 7.0, 37.0, 66.0, 96.0, 30.0, 80.0, 96.0, 80.0, 80.0, 66.0, 10.0, 37.0, 84.0, 100.0, 55.0, 23.0, 23.0, 13.0, 86.0, 80.0, 10.0, 95.0, 7.0, 80.0, 66.0, 66.0, 96.0, 32.0, 100.0, 47.0, 100.0, 7.0, 7.0, 96.0, 16.0, 100.0, 37.0, 37.0, 35.0, 37.0, 20.0, 4.0, 7.0, 32.0, 10.0, 66.0, 100.0, 66.0, 4.0, 37.0, 4.0, 16.0, 86.0, 52.0, 30.0, 45.0, 66.0, 93.0, 10.0, 45.0, 40.0, 13.0, 95.0, 45.0, 45.0, 13.0, 37.0, 47.0, 45.0, 37.0, 16.0, 35.0, 16.0, 93.0, 30.0, 32.0, 37.0, 32.0, 86.0, 23.0, 7.0, 7.0, 4.0, 86.0, 32.0, 66.0, 20.0, 66.0, 37.0, 37.0, 47.0, 7.0, 32.0, 16.0, 7.0, 86.0, 100.0, 84.0, 80.0, 95.0, 37.0, 37.0, 84.0, 13.0, 35.0, 86.0, 86.0, 100.0, 66.0, 4.0, 95.0, 37.0, 23.0, 84.0, 45.0

# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(test_data)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.

In [6]:
#要改
RegNetY008 =tf.keras.applications.regnet.RegNetY008(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
#要改
RegNetY008.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
#要改
x = RegNetY008(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 regnety008 (Functional)     (None, 7, 7, 768)         5524056   
                                                                 
 global_average_pooling2d (G  (None, 768)              0         
 lobalAveragePooling2D)                                          
                                                                 
 out_weight (Dense)          (None, 1)                 769       
                                                                 
Total params: 5,524,825
Trainable params: 5,494,937
Non-trainable params: 29,888
_________________________________________________________________


In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                    callbacks=[cp_callback])
#要改
np.savetxt('RegNetY008_loss_0.txt',history.history['loss'])
#要改
np.savetxt('RegNetY008_valloss_0.txt',history.history['val_loss'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Sat Apr 15 12:35:36 2023
Epoch 1/300
591/591 [==============================] - ETA: 0s - loss: 421.0701

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY008\0\RegNetY008.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY008\0\RegNetY008.ckpt\assets


591/591 [==============================] - 95s 135ms/step - loss: 421.0701 - val_loss: 13.5120
Epoch 2/300
591/591 [==============================] - ETA: 0s - loss: 6.6742

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY008\0\RegNetY008.ckpt\assets


...

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY008\0\RegNetY008.ckpt\assets


591/591 [==============================] - 101s 172ms/step - loss: 0.0298 - val_loss: 0.0327
Epoch 132/300
591/591 [==============================] - 84s 142ms/step - loss: 0.0292 - val_loss: 0.0454
Epoch 133/300
591/591 [==============================] - 84s 141ms/step - loss: 0.0466 - val_loss: 0.1923
Epoch 134/300
591/591 [==============================] - 83s 141ms/step - loss: 0.0275 - val_loss: 0.1240
Epoch 135/300
591/591 [==============================] - 84s 141ms/step - loss: 0.0442 - val_loss: 0.2979
Epoch 136/300
591/591 [==============================] - 83s 141ms/step - loss: 0.0207 - val_loss: 0.0492
Epoch 137/300
591/591 [==============================] - 83s 140ms/step - loss: 0.0225 - val_loss: 0.1645
Epoch 138/300
591/591 [==============================] - 83s 140ms/step - loss: 0.0265 - val_loss: 0.1195
Epoch 139/300
591/591 [==============================] - 82s 139ms/step - loss: 0.0211 - val_loss: 0.1087
Epoch 140/300
591/591 [==============================] - 83

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY008\0\RegNetY008.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY008\0\RegNetY008.ckpt\assets


591/591 [==============================] - 105s 178ms/step - loss: 0.0277 - val_loss: 0.0288
Epoch 181/300
591/591 [==============================] - 87s 146ms/step - loss: 0.0339 - val_loss: 0.1041
Epoch 182/300
591/591 [==============================] - 88s 148ms/step - loss: 0.0191 - val_loss: 0.0533
Epoch 183/300
591/591 [==============================] - 87s 147ms/step - loss: 0.0329 - val_loss: 0.2201
Epoch 184/300
591/591 [==============================] - 87s 148ms/step - loss: 0.0242 - val_loss: 0.1822
Epoch 185/300
591/591 [==============================] - 86s 146ms/step - loss: 0.0219 - val_loss: 0.1063
Epoch 186/300
591/591 [==============================] - 87s 148ms/step - loss: 0.0213 - val_loss: 0.1528
Epoch 187/300
591/591 [==============================] - 87s 147ms/step - loss: 0.0212 - val_loss: 0.0793
Epoch 188/300
591/591 [==============================] - 87s 147ms/step - loss: 0.0188 - val_loss: 0.1555
Epoch 189/300
591/591 [==============================] - 88